<a href="https://colab.research.google.com/github/simecek/dspracticum2020/blob/master/lecture_08/assignment/e_coli_promoters/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 13.3MB/s 


In [2]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Reshaping data from fasta to txt

In [3]:
classes = ['nonpromoters', 'promoters']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://raw.githubusercontent.com/simecek/dspracticum2020/master/lecture_08/assignment/e_coli_promoters/e_coli_{c}_{s}.fa", f"e_coli_{c}_{s}.fa")

In [4]:
for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

In [5]:
for c in classes:
    for s in sets:
        with open(f"e_coli_{c}_{s}.fa", "r") as fr:
            for record in SeqIO.parse(fr, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])


## Reading data

In [6]:
batch_size = 64

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 6791 files belonging to 2 classes.


In [8]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

Found 750 files belonging to 2 classes.


In [9]:
vectorize_layer = TextVectorization(output_mode='int')

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))

In [12]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)

## Model training

In [13]:
# one-hot encoding
onehot_layer = keras.layers.Lambda(lambda x: tf.one_hot(tf.cast(x,'int64'), 4))

In [24]:
model_type = "LSTM"

if model_type == "LSTM":
    model = tf.keras.Sequential([
          onehot_layer,
          keras.layers.LSTM(128, return_sequences=True),
          keras.layers.LSTM(128, return_sequences=False),
          keras.layers.Dense(1, activation="sigmoid")])                     
else:
    model = tf.keras.Sequential([
        onehot_layer,
        keras.layers.Conv1D(32, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Conv1D(16, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Conv1D(4, kernel_size=8, data_format='channels_last', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling1D(),
        keras.layers.Dropout(0.3),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(1, activation="sigmoid")])

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
epochs = 10
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
107/107 [==============================] - 4s 38ms/step - loss: 0.5392 - accuracy: 0.7265 - val_loss: 0.4849 - val_accuracy: 0.7880
Epoch 2/10
107/107 [==============================] - 3s 31ms/step - loss: 0.4761 - accuracy: 0.7862 - val_loss: 0.4316 - val_accuracy: 0.8027
Epoch 3/10
107/107 [==============================] - 3s 31ms/step - loss: 0.4491 - accuracy: 0.7963 - val_loss: 0.3983 - val_accuracy: 0.8333
Epoch 4/10
107/107 [==============================] - 3s 31ms/step - loss: 0.4139 - accuracy: 0.8201 - val_loss: 0.3931 - val_accuracy: 0.8200
Epoch 5/10
107/107 [==============================] - 3s 31ms/step - loss: 0.4004 - accuracy: 0.8314 - val_loss: 0.3514 - val_accuracy: 0.8613
Epoch 6/10
107/107 [==============================] - 4s 37ms/step - loss: 0.3937 - accuracy: 0.8284 - val_loss: 0.3756 - val_accuracy: 0.8400
Epoch 7/10
107/107 [==============================] - 4s 37ms/step - loss: 0.3604 - accuracy: 0.8451 - val_loss: 0.3371 - val_accuracy: 0.8667